In [138]:
# 라이브러리 로드
import pandas as pd
import numpy as np

In [139]:
# 데이터 로드

수익성 = pd.read_csv('./datasets/코스닥/코스닥 수익성.csv',encoding='cp949')
안정성 = pd.read_csv('./datasets/코스닥/코스닥 안정성.csv',encoding='cp949')
활동성 = pd.read_csv('./datasets/코스닥/코스닥 활동성.csv',encoding='cp949')
성장성 = pd.read_csv('./datasets/코스닥/코스닥 성장성 계산용.csv',encoding='cp949')
현금흐름 = pd.read_csv('./datasets/코스닥/코스닥 현금흐름 + 누적수익성비율.csv',encoding='cp949')
연구개발 = pd.read_csv('./datasets/코스닥/코스닥 rs.csv',encoding='cp949')
EBIT = pd.read_csv('./datasets/코스닥/에비~ 코스닥.csv',encoding='cp949')
per_pbr_총부채 = pd.read_csv('./datasets/코스닥/per pbr 총부채 코스닥.csv',encoding='cp949')
직원수_업력 = pd.read_csv('./datasets/코스닥/코스닥 직원수 급여 업력 외국인 대주주.csv',encoding='cp949')
시가총액 = pd.read_csv('./datasets/코스닥/코스닥시가총액.csv',index_col=0)  #### 시총도 붙여야됨
단기차입금 = pd.read_csv('./datasets/코스닥/코스닥 단기차입금 유동성 장기부채.csv',encoding='cp949')
거시변수 = pd.read_excel('./datasets/거시경제 변수/거시변수.xlsx', engine="openpyxl")
상장일 = pd.read_csv('./datasets/코스닥/코스닥 상장일.csv',encoding='cp949')
대분류 = pd.read_csv('./datasets/코스닥/코스닥 대분류.csv',encoding='cp949')
유형자산 = pd.read_csv("./datasets/코스닥/코스닥 유형자산증가율.csv", encoding= "cp949")

In [140]:
#  사용 안하는 코드 컬럼 삭제 
대분류.drop(['통계청 한국표준산업분류 코드 10차(대분류)'],axis=1,inplace=True)

In [141]:
#  데이터 결합

df = 안정성.merge(수익성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(활동성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(성장성,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(현금흐름,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(연구개발,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(EBIT,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(per_pbr_총부채,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(직원수_업력,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(단기차입금,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(상장일,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(대분류,on=['회사명','거래소코드','회계년도'],how='left')
df = df.merge(유형자산,on=['회사명','거래소코드','회계년도'],how='left')

In [142]:
len(df.columns)

97

In [143]:
## 거래소코드 패딩
def padding(x):
    x = str(x)
    return x.rjust(6, "0")
## 외국계기업 제거
def func_isnt_fcompany(x):
    if x[:1] == '9':
        return False
    else:
        return True

# 회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])

def func_not(x):
    return not x

def change_year_2(x):
    return (x[:4])

In [144]:
## 거래소 코드 패딩
df['거래소코드'] = df['거래소코드'].map(padding)
# 회계년도 제거
df['회계년도'] = df['회계년도'].map(change_year)

In [145]:
df.isna().sum()

회사명                            0
거래소코드                          0
회계년도                           0
자산(*)(요약)(IFRS)             1844
자본(*)(요약)(IFRS)             1844
                            ... 
유동성장기부채(요약)(IFRS연결)(백만원)    5530
상장일                            0
통계청 한국표준산업분류 10차(대분류)          0
유형자산의 증가(*)(IFRS)(천원)       1844
유형자산의 증가(*)(IFRS연결)(천원)     5530
Length: 97, dtype: int64

In [146]:
## 중복되는 데이터 제거
df.drop_duplicates(['거래소코드',"회계년도"], keep='last', inplace=True)
df.reset_index(drop=True, inplace=True)

## 없음


In [147]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 15617 개
전체 기업 수 1744


In [148]:
## 외국계 기업 제거
print("제거할 데이터 수 : ", len(df[df["거래소코드"].map(func_isnt_fcompany).map(func_not)]))
print('제거할 외국계 기업 수',len(df[~df["거래소코드"].map(func_isnt_fcompany)].회사명.unique()))

df = df[df["거래소코드"].map(func_isnt_fcompany)]
print('남은 데이터 수',df.shape[0]) # 외국계기업 데이터 24개 제거
print('남은 기업 수',len(df['회사명'].unique()))

제거할 데이터 수 :  152
제거할 외국계 기업 수 23
남은 데이터 수 15465
남은 기업 수 1721


In [149]:
## 회사수와 거래소코드수 차이가나는것 확인
print("회사수 : ", len(df["회사명"].unique()))
print("거래소코드수 : ", len(df["거래소코드"].unique()))

회사수 :  1721
거래소코드수 :  1743


In [150]:
## 거래소 코드 중복기업 전처리
list_overlap_code = list()
for i in list(df["회사명"].unique()):
    if len(df[df["회사명"]==i]["거래소코드"].unique()) > 1:
        list_overlap_code.append(i)
print("거래소코드가 2개인 기업 수", len(list_overlap_code), "개")

거래소코드가 2개인 기업 수 22 개


In [151]:
list_overlap_code

['(주)본느',
 '(주)세화피앤씨',
 '(주)아이엘사이언스',
 '(주)알로이스',
 '(주)에스에이티',
 '(주)엔케이맥스',
 '(주)원바이오젠',
 '(주)인산가',
 '(주)자비스',
 '(주)정다운',
 '(주)카이노스메드',
 '(주)티에스트릴리온',
 '(주)포인트엔지니어링',
 '(주)한국비엔씨',
 '(주)휴럼',
 '나무기술(주)',
 '소프트캠프(주)',
 '씨아이에스(주)',
 '에스케이오션플랜트(주)',
 '엠에프엠코리아(주)',
 '줌인터넷(주)',
 '패션플랫폼(주)']

In [152]:
# 중복 거래소코드 제거
def func_overlap_code(x):
    if x == (df[df["회사명"]==company_name]["거래소코드"].iloc[-1]) :
        return True
    else:
        return False

list_companyname = list(df["회사명"].unique())


dfs = list()


for i in list_companyname:
    company_name = i
    #각 기업별로 데이터프레임 생성
    df_concat = df[df["회사명"]==i]
    df_concat = df_concat[df_concat["거래소코드"].map(func_overlap_code)] # 거래소코드 바뀌기 전의 거래소코드의 데이터는 삭제
    dfs.append(df_concat)
df = pd.concat(dfs)
df.reset_index(drop=True, inplace=True)

In [153]:
# 회사명이 2개인 거래소코드 존재 확인 
list_overlap_code = list()
for i in list(df["거래소코드"].unique()):
    if len(df[df["거래소코드"]==i]["회사명"].unique()) > 1:
        list_overlap_code.append(i)
list_overlap_code

[]

In [154]:
## 연결 결측치 개별로 채우기
for i in range(df.shape[0]):
    if pd.isnull(df.loc[i,'유보율(IFRS연결)(백만원)']):
       df.loc[i,'유보율(IFRS연결)(백만원)'] = df.loc[i,'유보율(IFRS)']
    if pd.isnull(df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'자산(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'부채(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'부채(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]자본(*)(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]자본(*)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'당기순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'당기순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] = df.loc[i,'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융원가(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융원가(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융원가(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'금융수익(요약)(IFRS연결)(백만원)']):
       df.loc[i,'금융수익(요약)(IFRS연결)(백만원)'] = df.loc[i,'금융수익(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'매출액(수익)(요약)(IFRS연결)(백만원)'] = df.loc[i,'매출액(수익)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'연구개발비(IFRS연결)(천원)']):
       df.loc[i,'연구개발비(IFRS연결)(천원)'] = df.loc[i,'연구개발비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)']):
       df.loc[i,'[제조]      유형자산(요약)(IFRS연결)(백만원)'] = df.loc[i,'[제조]      유형자산(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'자본금회전률(IFRS연결)']):
       df.loc[i,'자본금회전률(IFRS연결)'] = df.loc[i,'자본금회전률(IFRS)']
    if pd.isnull(df.loc[i,'총자본회전률(IFRS연결)']):
       df.loc[i,'총자본회전률(IFRS연결)'] = df.loc[i,'총자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'재고자산회전률(IFRS연결)']):
       df.loc[i,'재고자산회전률(IFRS연결)'] = df.loc[i,'재고자산회전률(IFRS)']
    if pd.isnull(df.loc[i,'유형자산회전율(IFRS연결)']):
       df.loc[i,'유형자산회전율(IFRS연결)'] = df.loc[i,'유형자산회전율(IFRS)']
    if pd.isnull(df.loc[i,'자기자본회전률(IFRS연결)']):
       df.loc[i,'자기자본회전률(IFRS연결)'] = df.loc[i,'자기자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'타인자본회전률(IFRS연결)']):
       df.loc[i,'타인자본회전률(IFRS연결)'] = df.loc[i,'타인자본회전률(IFRS)']
    if pd.isnull(df.loc[i,'경영자본순이익률(IFRS연결)']):
       df.loc[i,'경영자본순이익률(IFRS연결)'] = df.loc[i,'경영자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'금융비용부담률(IFRS연결)']):
       df.loc[i,'금융비용부담률(IFRS연결)'] = df.loc[i,'금융비용부담률(IFRS)']
    if pd.isnull(df.loc[i,'총자본순이익률(IFRS연결)']):
       df.loc[i,'총자본순이익률(IFRS연결)'] = df.loc[i,'총자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'매출액총이익률(IFRS연결)']):
       df.loc[i,'매출액총이익률(IFRS연결)'] = df.loc[i,'매출액총이익률(IFRS)']
    if pd.isnull(df.loc[i,'자기자본순이익률(IFRS연결)']):
       df.loc[i,'자기자본순이익률(IFRS연결)'] = df.loc[i,'자기자본순이익률(IFRS)']
    if pd.isnull(df.loc[i,'총자본정상영업이익률(IFRS연결)']):
       df.loc[i,'총자본정상영업이익률(IFRS연결)'] = df.loc[i,'총자본정상영업이익률(IFRS)']
    if pd.isnull(df.loc[i,'자산(*)(요약)(IFRS연결)']):
       df.loc[i,'자산(*)(요약)(IFRS연결)'] = df.loc[i,'자산(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'자본(*)(요약)(IFRS연결)']):
       df.loc[i,'자본(*)(요약)(IFRS연결)'] = df.loc[i,'자본(*)(요약)(IFRS)']
    if pd.isnull(df.loc[i,'현금비율(IFRS연결)(백만원)']):
       df.loc[i,'현금비율(IFRS연결)(백만원)'] = df.loc[i,'현금비율(IFRS)']
    if pd.isnull(df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']):
       df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'] = df.loc[i,'* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'차입금의존도(IFRS연결)']):
       df.loc[i,'차입금의존도(IFRS연결)'] = df.loc[i,'차입금의존도(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'감가상각비(IFRS연결)(천원)']):
       df.loc[i,'감가상각비(IFRS연결)(천원)'] = df.loc[i,'감가상각비(IFRS)(천원)']
    if pd.isnull(df.loc[i,'부채비율(IFRS연결)']):
       df.loc[i,'부채비율(IFRS연결)'] = df.loc[i,'부채비율(IFRS)']
    if pd.isnull(df.loc[i,'당좌비율(IFRS연결)']):
       df.loc[i,'당좌비율(IFRS연결)'] = df.loc[i,'당좌비율(IFRS)']
    if pd.isnull(df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)']):
       df.loc[i,'*이자비용(요약)(IFRS연결)(백만원)'] = df.loc[i,'*이자비용(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'순운전자본비율(IFRS연결)']):
       df.loc[i,'순운전자본비율(IFRS연결)'] = df.loc[i,'순운전자본비율(IFRS)']
    if pd.isnull(df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)']):
       df.loc[i,'유동성장기부채(요약)(IFRS연결)(백만원)'] = df.loc[i,'유동성장기부채(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)']):
       df.loc[i,'단기차입금(요약)(IFRS연결)(백만원)'] = df.loc[i,'단기차입금(요약)(IFRS)(백만원)']
    if pd.isnull(df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)']):
       df.loc[i,'유형자산의 증가(*)(IFRS연결)(천원)'] = df.loc[i,'유형자산의 증가(*)(IFRS)(천원)']


In [155]:
# #  연결 재무제표를 개별제무제표로 대체
# 위의 코드 요약해서 다시 세워본 코드 


# connected_cols = [col for col in df.columns if "연결" in col]

# for connected_col in connected_cols:
#     original_col = connected_col.replace("연결", "")
#     if original_col in df.columns:
#         df.loc[df[connected_col].isnull(), connected_col] = df[original_col]

In [156]:
#  삭제 할 개별재무제표 컬럼 정리  

drop_col = ['유보율(IFRS)' ,'[제조]* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)','[제조]      이익잉여금(결손금)(요약)(IFRS)(백만원)',
'[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS)(백만원)', 
'부채(*)(요약)(IFRS)(백만원)','[제조]자본(*)(요약)(IFRS)(백만원)','당기순이익(손실)(요약)(IFRS)(백만원)', 
'법인세비용차감전순이익(손실)(요약)(IFRS)(백만원)','금융원가(요약)(IFRS)(백만원)','금융수익(요약)(IFRS)(백만원)',
'매출액(수익)(요약)(IFRS)(백만원)','연구개발비(IFRS)(천원)',
'[제조]      유형자산(요약)(IFRS)(백만원)','자본금회전률(IFRS)',
'총자본회전률(IFRS)','재고자산회전률(IFRS)','유형자산회전율(IFRS)','자기자본회전률(IFRS)','타인자본회전률(IFRS)',
'경영자본순이익률(IFRS)','금융비용부담률(IFRS)','총자본순이익률(IFRS)','매출액총이익률(IFRS)',
'자기자본순이익률(IFRS)','총자본정상영업이익률(IFRS)','자산(*)(요약)(IFRS)','자본(*)(요약)(IFRS)',
 '현금비율(IFRS)','* (정상)영업손익(보고서기재)(요약)(IFRS)(백만원)',
 '차입금의존도(IFRS)(백만원)','감가상각비(IFRS)(천원)','부채비율(IFRS)',
'당좌비율(IFRS)','*이자비용(요약)(IFRS)(백만원)','순운전자본비율(IFRS)',
'[제조]자산(*)(요약)(IFRS)(백만원)_x','[제조]자산(*)(요약)(IFRS연결)(백만원)_x',
'[제조]자산(*)(요약)(IFRS)(백만원)_y','[제조]자산(*)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_y', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_y',
'[제조]매출액(수익)(요약)(IFRS)(백만원)_x', '[제조]매출액(수익)(요약)(IFRS연결)(백만원)_x',
'유동성장기부채(요약)(IFRS)(백만원)','단기차입금(요약)(IFRS)(백만원)','자산(*)(요약)(IFRS연결)(백만원)','[제조]자본(*)(요약)(IFRS연결)(백만원)',"유형자산의 증가(*)(IFRS)(천원)"
]

# 개별재무제표 컬럼 삭제
df.drop(drop_col,axis=1,inplace=True)

In [157]:
len(df.columns)

48

In [158]:
## 성장성 지표인 증가율 컬럼 만들기
성장성_col = ['[제조]      유형자산(요약)(IFRS연결)(백만원)','매출액(수익)(요약)(IFRS연결)(백만원)',
           '자본(*)(요약)(IFRS연결)','자산(*)(요약)(IFRS연결)',
           '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)']

## 증가율 컬럼 전기에 음수일때 당기에 음수지만 증가할때 증가율 계산
## ex (2011년 -20, 2012년 -10) -> -10-(-20) / np.abs(-20)= 
증가율_df = (df.groupby(['회사명', '거래소코드'])[성장성_col].shift(0) - df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])[성장성_col].shift(1))*100
증가율_df.rename(columns={'[제조]      유형자산(요약)(IFRS연결)(백만원)':'유형자산증가율', '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액증가율',
       '자본(*)(요약)(IFRS연결)':'자기자본증가율', '자산(*)(요약)(IFRS연결)':'총자본증가율',
       '[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'정상영업이익증가율'},inplace=True)

df.drop(['[제조]      유형자산(요약)(IFRS연결)(백만원)','[제조]* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)'],axis=1,inplace=True)
df = pd.concat([df,증가율_df],axis=1)

In [159]:
## 시가총액 merge
시가총액['회계년도'] = 시가총액['회계년도'].astype(str).str[:4].astype(int)
df = df.merge(시가총액,on=['거래소코드','회계년도'],how='left')
df['시가총액(백만원)'] = df['시가총액'] / 1000000
df.drop(['시가총액'],axis=1,inplace=True)

## 거시변수 merge
거시변수.rename(columns={'회계연도':'회계년도'},inplace=True)
df = df.merge(거시변수,on=['회계년도'],how='left')

In [160]:
### 파생변수 만들기 


## 기업별 업력 설립일 이후 지속된 기간
df['업력'] = df['회계년도'] -  pd.to_datetime(df['설립일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)
df.drop('설립일',axis=1,inplace=True)

## 누적수익성비율  - 자산0이면 inf - 공시안함 
df['누적수익성비율'] = df['[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)']/df['자산(*)(요약)(IFRS연결)']

## 이자부담율
df['이자부담률'] = df['EBITDA(IFRS)(백만원)'] / df['*이자비용(요약)(IFRS연결)(백만원)'] 

## 부채상환계수 EBITDA / (이자비용 + 단기차입금 + 유동성장기부채)
df['부채상환계수'] = df['EBITDA(IFRS)(백만원)'] / (df['*이자비용(요약)(IFRS연결)(백만원)']+df['단기차입금(요약)(IFRS연결)(백만원)'] + df['유동성장기부채(요약)(IFRS연결)(백만원)'])

## 현금흐름_자산 = 영업현금흐름 / 당기총자산
df['현금흐름 대 자산'] = df.groupby(['회사명','거래소코드'])['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'].shift(0) / df.groupby(['회사명','거래소코드'])['자산(*)(요약)(IFRS연결)'].shift(0)

## 현금흐름_매출액
df['현금흐름 대 매출액'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

# 토빈Q
df['토빈Q'] = (df['부채(*)(요약)(IFRS연결)(백만원)'] + df['시가총액(백만원)']) / df['자산(*)(요약)(IFRS연결)']

## TMD
df['TMD'] = df['부채(*)(요약)(IFRS연결)(백만원)'] / df['시가총액(백만원)']

## 알트만 변수
df['시총/총자산'] = df['시가총액(백만원)'] / df['자산(*)(요약)(IFRS연결)']

## RD -단위도 수정
df['연구개발비(IFRS연결)(백만원)'] = df['연구개발비(IFRS연결)(천원)']/1000
df.drop(['연구개발비(IFRS연결)(천원)'],axis=1,inplace=True)
df['RD'] = df['연구개발비(IFRS연결)(백만원)'] / df['매출액(수익)(요약)(IFRS연결)(백만원)']

## 직원수변동률 , 직원인건비총액(백만원단위) (고민필요)
df['직원수변동률%'] = df.groupby(['회사명','거래소코드'])['합계_기말인원(명)'].pct_change()*100

## 직원인건비총액(백만원)
df['직원인건비총액(백만원)'] = (df['합계_년간총급여(천원)']/1000) * df['합계_기말인원(명)']
df.drop(['합계_년간총급여(천원)'],axis=1,inplace=True)

## 알트만 변수  EBIT = 법인세비용차감전순이익+금융원가-금융수익
df['EBIT/총자산'] = (df['법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)'] + df['금융원가(요약)(IFRS연결)(백만원)']-df['금융수익(요약)(IFRS연결)(백만원)'])/df['자산(*)(요약)(IFRS연결)']

## PBR 
df['PBR'] = df['시가총액(백만원)']  / df['자본(*)(요약)(IFRS연결)']


# 영업손익 전기영업손익 + 당기영업손익
df["당기전기영업손익"] = (df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(0) + df.groupby(["회사명", "거래소코드"])["* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)"].shift(1)) / 2

# capex컬럼 단위 맞춰주기
df["유형자산의 증가(*)(IFRS연결)(천원)"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] / 1000

# findep =  (capex - 영업현금흐름 ) / capex
df["FINDEP"] = df["유형자산의 증가(*)(IFRS연결)(천원)"] - df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)']

# PBR 변화율
df['PBR변화율']  = (df.groupby(['회사명', '거래소코드'])['PBR'].shift(0) - df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['PBR'].shift(1)) *100

# ROA 변화율
df['ROA변화율']  = (df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(0) - df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) / np.abs(df.groupby(['회사명', '거래소코드'])['총자본순이익률(IFRS연결)'].shift(1)) *100

# EBITDA마진율
df["EBITDA마진율"] = (df["EBITDA(IFRS)(백만원)"] / df["매출액(수익)(요약)(IFRS연결)(백만원)"]) * 100

# 논문을 바탕으로 새로운 컬럼 생성

df['현금흐름/총부채비율'] = df['[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)'] / df['부채(*)(요약)(IFRS연결)(백만원)']
df["영업현금흐름-단기차입금"] = df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["단기차입금(요약)(IFRS연결)(백만원)"]
df["abs(영업현금흐름-당기순이익)/매출액"] = np.abs((df["[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)"] - df["당기순이익(손실)(요약)(IFRS연결)(백만원)"])) / df["매출액(수익)(요약)(IFRS연결)(백만원)"]



In [161]:
len(df.columns)

74

In [162]:
# 컬럼이름 정리

rename_list = {'자산(*)(요약)(IFRS연결)':'자산', '자본(*)(요약)(IFRS연결)':'자본',
       '차입금의존도(IFRS연결)':'차입금의존도', '유보율(IFRS연결)(백만원)':'유보율', '현금비율(IFRS연결)(백만원)':'현금비율',
       '부채비율(IFRS연결)':'부채비율', '당좌비율(IFRS연결)':'당좌비율', '순운전자본비율(IFRS연결)':'순운전자본비율',
       '*이자비용(요약)(IFRS연결)(백만원)':'이자비용', '* (정상)영업손익(보고서기재)(요약)(IFRS연결)(백만원)':'영업손익',
       '감가상각비(IFRS연결)(천원)':'감가상각비', 'EBITDA(IFRS)(백만원)':'EBITDA', '총자본정상영업이익률(IFRS연결)':'총자본정상영업이익률',
       '자기자본순이익률(IFRS연결)':'자기자본순이익률', '총자본순이익률(IFRS연결)':'총자본순이익률', '매출액총이익률(IFRS연결)':'매출액총이익률',
       '금융비용부담률(IFRS연결)':'금융비용부담률', '경영자본순이익률(IFRS연결)':'경영자본순이익률', '자본금회전률(IFRS연결)':'자본금회전률',
       '유형자산회전율(IFRS연결)':'유형자산회전율', '재고자산회전률(IFRS연결)':'재고자산회전률', '자기자본회전률(IFRS연결)':'자기자본회전률',
       '타인자본회전률(IFRS연결)':'타인자본회전률', '총자본회전률(IFRS연결)':'총자본회전률', 
       '[제조]      이익잉여금(결손금)(요약)(IFRS연결)(백만원)':'이익잉여금',
       '[제조]영업활동으로 인한 현금흐름(간접법)(*)(요약)(IFRS연결)(백만원)':'영업활동으로 인한 현금흐름',
       '매출액(수익)(요약)(IFRS연결)(백만원)':'매출액', '금융원가(요약)(IFRS연결)(백만원)':'금융원가',
       '금융수익(요약)(IFRS연결)(백만원)':'금융수익', '법인세비용차감전순이익(손실)(요약)(IFRS연결)(백만원)':'법인세비용차감전순이익',
       '당기순이익(손실)(요약)(IFRS연결)(백만원)':'당기순이익', 
       '부채(*)(요약)(IFRS연결)(백만원)':'부채', '외국인_주식분포비율':'외국인지분율', '보통주식비율':'대주주지분율',
       '단기차입금(요약)(IFRS연결)(백만원)':'단기차입금', '유동성장기부채(요약)(IFRS연결)(백만원)':'유동성장기부채',
       '시가총액(백만원)':'시가총액','연구개발비(IFRS연결)(백만원)':'연구개발비','직원인건비총액(백만원)':'직원인건비총액','통계청 한국표준산업분류 10차(대분류)':'산업군',
       "유형자산의 증가(*)(IFRS연결)(천원)" : "CAPEX"
       }

df.rename(columns=rename_list,inplace=True)

In [163]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '차입금의존도', '유보율', '현금비율', '부채비율',
       '당좌비율', '순운전자본비율', '이자비용', '영업손익', '감가상각비', 'EBITDA', '총자본정상영업이익률',
       '자기자본순이익률', '총자본순이익률', '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률',
       '유형자산회전율', '재고자산회전률', '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금',
       '영업활동으로 인한 현금흐름', '매출액', '금융원가', '금융수익', '법인세비용차감전순이익', '당기순이익', '부채',
       '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금', '유동성장기부채', '상장일', '산업군',
       'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율', '정상영업이익증가율', '시가총액',
       '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수', '현금흐름 대 자산',
       '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD', '직원수변동률%',
       '직원인건비총액', 'EBIT/총자산', 'PBR', '당기전기영업손익', 'FINDEP', 'PBR변화율', 'ROA변화율',
       'EBITDA마진율', '현금흐름/총부채비율', '영업현금흐름-단기차입금', 'abs(영업현금흐름-당기순이익)/매출액'],
      dtype='object')

In [164]:
## 회계년도 2011년도 제거
df['상장일'] = pd.to_datetime(df['상장일'],format='%Y/%m/%d').dt.strftime('%Y').astype(int)


df = df[df['회계년도']>2011].reset_index(drop=True)

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))


전체 행: 14077 개
전체 기업 수 1684


In [165]:
df.isna().sum()

회사명                         0
거래소코드                       0
회계년도                        0
자산                       1456
자본                       1456
                         ... 
ROA변화율                   2271
EBITDA마진율                1645
현금흐름/총부채비율               1457
영업현금흐름-단기차입금             1456
abs(영업현금흐름-당기순이익)/매출액    1459
Length: 74, dtype: int64

-----

## 타겟 라벨링
-----


In [166]:
#  데이터 로드, 필요 없는 컬럼 삭제

부실 = pd.read_csv('./datasets/y/y값최종.csv',index_col=0)
부실['회계년도'] = 부실['회계년도'].astype(int)
부실['거래소코드'] = 부실['거래소코드'].map(padding)
부실.drop(['번호','회사명'],axis=1,inplace=True)

In [167]:
# 데이터 결합
df = df.merge(부실,on=['회계년도','거래소코드'],how='left')

In [168]:
# 타겟 라벨링 

df["공시제목"].fillna(0,inplace=True)

ind_1 = df[df["공시제목"] != 0].index

### set1 (상폐: T-2,   회생 : T-2)
df.loc[ind_1,"target_1"] = 1

for i in df.index:
    if (df.loc[i,"target_1"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) :
        df.loc[i-1, "target_1"] = 1
df['target_1'].fillna(0,inplace=True)
df['target_1'] = df['target_1'].astype(int)

# ### set2  (상폐 : T-3, 회생: T-1)
df.loc[ind_1,"target_2"] = 1

for i in df.index:
    if (df.loc[i,"target_2"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업') :
        df.loc[i-1, "target_2"] = 1 
    if (df.loc[i,"target_2"] == 1) and (df.loc[i-1,"회사명"] == df.loc[i-2,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-2, "target_2"] = 1

df['target_2'].fillna(0,inplace=True)
df['target_2'] = df['target_2'].astype(int)

## set_3 (상폐 : T-2, 회생 : T-1)

df.loc[ind_1,"target_3"] = 1

for i in df.index:
    if (df.loc[i,"target_3"] == 1) and (df.loc[i,"회사명"] == df.loc[i-1,"회사명"]) and (df.loc[i,'공시제목']=='상장폐지기업'):
        df.loc[i-1, "target_3"] = 1
df['target_3'].fillna(0,inplace=True)
df['target_3'] = df['target_3'].astype(int)


In [169]:
print(df.target_1.sum())
print(df.target_2.sum())
print(df.target_3.sum())

312
389
275


In [170]:
def get_index_with_50_or_more_nan_columns(df):
    
    # 결측치 컬럼 개수 계산
    nan_column_counts = df.isnull().sum(axis=1)
    
    # 결측치 컬럼 개수가 50개 이상인 인덱스 번호 추출
    index_with_50_or_more_nan_columns = nan_column_counts[nan_column_counts >= 50].index.to_list()
    print('제거행 수:' , len(index_with_50_or_more_nan_columns))

    df.drop(index=index_with_50_or_more_nan_columns,inplace=True)
    return df


In [171]:
len(get_index_with_50_or_more_nan_columns(df))

제거행 수: 1455


12622

In [172]:
print('부실데이터 수:',df[df['target_1']==1].shape[0])
print('부실데이터 수:',df[df['target_2']==1].shape[0])
print('부실데이터 수:',df[df['target_3']==1].shape[0])

부실데이터 수: 279
부실데이터 수: 344
부실데이터 수: 245


In [173]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                         0
거래소코드                       0
회계년도                        0
자산                          1
자본                          1
차입금의존도                    186
유보율                       186
현금비율                      186
부채비율                      186
당좌비율                      186
순운전자본비율                   186
이자비용                        1
영업손익                        1
감가상각비                       1
EBITDA                    188
총자본정상영업이익률                186
자기자본순이익률                  186
총자본순이익률                   186
매출액총이익률                   186
금융비용부담률                   186
경영자본순이익률                  186
자본금회전률                    186
유형자산회전율                   186
재고자산회전률                   186
자기자본회전률                   186
타인자본회전률                   186
총자본회전률                    186
이익잉여금                       1
영업활동으로 인한 현금흐름              1
매출액                         1
금융원가                        1
금융수익                        1
법인세비용차감전순이익                 1
당기순이익     

In [174]:
# 상장일이 회계년도 보다 작은 데이터 삭제
df = df[df["회계년도"] > df["상장일"]]
df.reset_index(drop = True, inplace = True)

print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))

전체 행: 10807 개
전체 기업 수 1473


In [175]:
# 이용한 컬럼 삭제 

df.drop(['자산','이자비용','매출액','부채','금융원가','시가총액',"EBITDA",'공시제목','상장일'],axis=1,inplace=True)

In [176]:
# 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "이자부담률"] = -5.995505908521291

# 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "부채상환계수"] = -760.5303077054485

# 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 
df.loc[1879, "유형자산증가율"] = 0                    # # 유형자산을 모두 처분하여 감가상각비가 없어 EBITDA값이 결측치인 14년도 영진코퍼레이션 결측치에 영업이익값으로 대체


# 이자부당률이 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "이자부담률"] = -5.995505908521291

# 부채상환계수rk 결측치로 나오는 14년도 영진코퍼레이션 결측치 직접 값 계산해서 결측치 대체

df.loc[4173, "부채상환계수"] = -760.5303077054485

# 대주주 지분율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[903, "대주주지분율"] = 50                    # 드림티엔터테인먼트
df.loc[2913, "대주주지분율"] = 27.83                # 아라온테크
df.loc[3668, "대주주지분율"] = 49.8                 # 에이루트
df.loc[3702, "대주주지분율"] = 18.5                 # 에이제이에스
df.loc[4399, "대주주지분율"] = 23.84                # 와이즈파워
df.loc[5542, "대주주지분율"] = 100                  # 제이테크놀로지
df.loc[8155, "대주주지분율"] = 12.03                # 대한그린파워
df.loc[8326, "대주주지분율"] = 14.94                # 디브이에스코리아
df.loc[8807, "대주주지분율"] = 40.78                # 세미텍


#  유형자산증가율 없는 컬럼 다트에서 값 찾아서 대체 

df.loc[1879, "유형자산증가율"] = 0                    # 비츠로시스
df.loc[1880, "유형자산증가율"] = 0                    # 비츠로시스

# 외국인 지분율 결측치 0으로 대체
df["외국인지분율"].fillna(0, inplace=True)


# 직원인건비총액 결측치 0으로 대체
df["직원인건비총액"].fillna(0, inplace=True)

# 합계_기말인원(명) 결측치 0으로 대체
df["합계_기말인원(명)"].fillna(0, inplace=True)



# #유형자산증가율 없는 컬럼 삭제

de_ind_1 = df[df[["매출액증가율"]].isna().any(1)].index
df.drop(index=de_ind_1, axis=0, inplace=True)
 # 메드팩토 셀리드 큐리언트


 # 증가율 컬럼 당해 회계년도에 상장하여 전년도 자료가 없어 결측치로 뜨는 데이터 삭제   
de_ind = df[df[["총자본증가율"]].isna().any(1)].index
df.drop(index=de_ind, axis=0, inplace=True)

# 직원수변동률 결측치 0으로 대체 
de_ind_2 = df[df[["직원수변동률%"]].isna().any(1)].index
df.drop(index=de_ind_2, axis=0, inplace=True)
#  비덴트 썬테크놀로지스

In [177]:
pd.set_option('display.max_rows',None)
print(df.isna().sum())
pd.reset_option('display.max_rows',None)

회사명                       0
거래소코드                     0
회계년도                      0
자본                        0
차입금의존도                    0
유보율                       0
현금비율                      0
부채비율                      0
당좌비율                      0
순운전자본비율                   0
영업손익                      0
감가상각비                     0
총자본정상영업이익률                0
자기자본순이익률                  0
총자본순이익률                   0
매출액총이익률                   0
금융비용부담률                   0
경영자본순이익률                  0
자본금회전률                    0
유형자산회전율                   0
재고자산회전률                   0
자기자본회전률                   0
타인자본회전률                   0
총자본회전률                    0
이익잉여금                     0
영업활동으로 인한 현금흐름            0
금융수익                      0
법인세비용차감전순이익               0
당기순이익                     0
합계_기말인원(명)                0
외국인지분율                    0
대주주지분율                    0
단기차입금                     0
유동성장기부채                   0
산업군                       0
CAPEX               

In [178]:
# 결측치 제거(22개)
df.dropna(axis=0, inplace=True)

In [179]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10662 개
전체 기업 수 1453
결측치 수 :  0


In [180]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자본', '차입금의존도', '유보율', '현금비율', '부채비율', '당좌비율',
       '순운전자본비율', '영업손익', '감가상각비', '총자본정상영업이익률', '자기자본순이익률', '총자본순이익률',
       '매출액총이익률', '금융비용부담률', '경영자본순이익률', '자본금회전률', '유형자산회전율', '재고자산회전률',
       '자기자본회전률', '타인자본회전률', '총자본회전률', '이익잉여금', '영업활동으로 인한 현금흐름', '금융수익',
       '법인세비용차감전순이익', '당기순이익', '합계_기말인원(명)', '외국인지분율', '대주주지분율', '단기차입금',
       '유동성장기부채', '산업군', 'CAPEX', '유형자산증가율', '매출액증가율', '자기자본증가율', '총자본증가율',
       '정상영업이익증가율', '장단기금리차', 'cpi', '업력', '누적수익성비율', '이자부담률', '부채상환계수',
       '현금흐름 대 자산', '현금흐름 대 매출액', '토빈Q', 'TMD', '시총/총자산', '연구개발비', 'RD',
       '직원수변동률%', '직원인건비총액', 'EBIT/총자산', 'PBR', '당기전기영업손익', 'FINDEP', 'PBR변화율',
       'ROA변화율', 'EBITDA마진율', '현금흐름/총부채비율', '영업현금흐름-단기차입금',
       'abs(영업현금흐름-당기순이익)/매출액', 'target_1', 'target_2', 'target_3'],
      dtype='object')


## 결측치 처리 완료
---

In [181]:
# 무한대 값 확인

print("양의 무한대 :",df[df[["이자부담률"]].isin([np.inf]).any(1)]["target_3"].value_counts())
print("음의 무한대 :", df[df[["이자부담률"]].isin([-np.inf]).any(1)]["target_3"].value_counts())

양의 무한대 : 0    548
1      1
Name: target_3, dtype: int64
음의 무한대 : 0    96
1     3
Name: target_3, dtype: int64


In [182]:
# 이자부담률 무한대 값 삭제
# 전년도 이자부담률 데이터가 0값이서 데이터 제거(648개)

a = df[df["이자부담률"].isin([np.inf])].index.to_list()
b = df[df["이자부담률"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [183]:
# 유형자산증가율 무한대 값 삭제
# 전년도 유형자산 데이터가 0값이서 데이터 제거(2개)


a = df[df["유형자산증가율"].isin([np.inf])].index.to_list()
b = df[df["유형자산증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [184]:
# 현금흐름 대 매출액 무한대 값 삭제
# 전년도 매출액 데이터가 0값이서 데이터 제거(2개)

a = df[df["현금흐름 대 매출액"].isin([np.inf])].index.to_list()
b = df[df["현금흐름 대 매출액"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [185]:
# 매출액증가율 무한대 값 삭제
# 전년도 매출액 데이터가 0값이서 데이터 제거

a = df[df["매출액증가율"].isin([np.inf])].index.to_list()
b = df[df["매출액증가율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [186]:
# ROA변화율 무한대 값 삭제
# 전년도 ROA 데이터가 0값이서 데이터 제거

a = df[df["ROA변화율"].isin([np.inf])].index.to_list()
b = df[df["ROA변화율"].isin([-np.inf])].index.to_list()
c = a + b
df.drop(index=c, inplace=True)

In [187]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 10003 개
전체 기업 수 1442
결측치 수 :  0



## 무한대 값 처리 완료
---

In [188]:
# 산업군이 "금융 및 보험업"인 데이터 삭제

df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_3", "산업군"]]

,회사명,회계년도,target_3,산업군
7924,글로벌텍스프리(주),2018,0,금융 및 보험업
7925,글로벌텍스프리(주),2019,0,금융 및 보험업
7926,글로벌텍스프리(주),2020,0,금융 및 보험업
7927,글로벌텍스프리(주),2021,0,금융 및 보험업
10699,해성산업(주),2018,0,금융 및 보험업
10700,해성산업(주),2019,0,금융 및 보험업
10701,해성산업(주),2020,0,금융 및 보험업
10702,해성산업(주),2021,0,금융 및 보험업


In [189]:
del_col = df[df["산업군"].str.contains("금융")][["회사명", "회계년도", "target_3", "산업군"]].index

df.drop(index=del_col, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)


In [190]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9995 개
전체 기업 수 1440
결측치 수 :  0


# 원본 데이터 결측치 처리 완료
---

In [192]:
# 타겟 1인 기업들 0값 지워주기

# 부실정의 컬럼 
df['sum'] = df['target_1']+df['target_2']+df['target_3']

## 임시 부실데이터저장
temp = pd.DataFrame()
for i in df['회사명'].unique():
    a = df[(df['회사명']==i) & df['sum']!=0]
    temp = pd.concat([temp,a],axis=0)
temp.drop(columns=['sum'],axis=1,inplace=True)
print(temp.shape)



(361, 69)


In [193]:
for i in temp['회사명'].unique():
    df = df[~(df['회사명']==i)]

df = pd.concat([df,temp],axis=0)

In [194]:
df.drop(columns=["sum"], axis=1, inplace=True)
df.reset_index(drop=True,inplace=True)
print(df.shape)


(9502, 69)


In [195]:
df["target_3"].value_counts()

0    9265
1     237
Name: target_3, dtype: int64

In [196]:
print(len(df[df["target_3"]==1]["회사명"].unique()))
print(len(df[df["target_3"]==0]["회사명"].unique()))

147
1403


In [197]:
print('전체 행:', df.shape[0],'개')
print('전체 기업 수',len(df['회사명'].unique()))
print("결측치 수 : ", df.isna().sum().sum())

전체 행: 9502 개
전체 기업 수 1440
결측치 수 :  0


In [198]:
df.describe()

,회계년도,자본,차입금의존도,유보율,현금비율,부채비율,당좌비율,순운전자본비율,영업손익,감가상각비,총자본정상영업이익률,자기자본순이익률,총자본순이익률,매출액총이익률,금융비용부담률,경영자본순이익률,자본금회전률,유형자산회전율,재고자산회전률,자기자본회전률,타인자본회전률,총자본회전률,이익잉여금,영업활동으로 인한 현금흐름,금융수익,법인세비용차감전순이익,당기순이익,합계_기말인원(명),외국인지분율,대주주지분율,단기차입금,유동성장기부채,CAPEX,유형자산증가율,매출액증가율,자기자본증가율,총자본증가율,정상영업이익증가율,장단기금리차,cpi,업력,누적수익성비율,이자부담률,부채상환계수,현금흐름 대 자산,현금흐름 대 매출액,토빈Q,TMD,시총/총자산,연구개발비,RD,직원수변동률%,직원인건비총액,EBIT/총자산,PBR,당기전기영업손익,FINDEP,PBR변화율,ROA변화율,EBITDA마진율,현금흐름/총부채비율,영업현금흐름-단기차입금,abs(영업현금흐름-당기순이익)/매출액,target_1,target_2,target_3
count,9502.000000,9.502000e+03,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9.502000e+03,9.502000e+03,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9.502000e+03,9.502000e+03,9502.000000,9.502000e+03,9.502000e+03,9502.000000,9502.000000,9502.000000,9.502000e+03,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9502.000000,9.502000e+03,9502.000000,9502.000000,9.502000e+03,9.502000e+03,9502.000000,9502.000000,9502.000000,9502.000000,9.502000e+03,9502.000000,9502.000000,9502.000000,9502.000000
mean,2017.008419,1.164249e+05,22.109065,1209.858890,69.913561,122.157084,226.347779,19.569955,9.109636e+03,1.311599e+06,2.055726,-6.514244,-0.963739,22.154131,2.366389,-1.660542,20.460046,13.407536,216.897838,1.733837,2.513388,0.836118,4.848065e+04,1.047259e+04,2947.015470,6.732894e+03,4.609365e+03,251.374237,4.874325,36.297749,2.851653e+04,6881.100926,10724.100997,47.454760,28.877430,13.784187,11.893570,-138.892308,0.998804,97.404936,23.480530,0.099797,189.444899,130.901165,0.033315,-0.239428,1.652664,0.965291,1.232441,2742.897549,0.224814,4.650872,8.136112e+06,-0.000018,2.238370,8.790060e+03,2.515128e+02,23.617844,-110.704429,-18.970841,0.141128,-1.804394e+04,0.269012,0.028310,0.034624,0.024942
std,2.883620,2.084800e+05,19.226879,1664.170987,147.486871,777.676467,357.376604,29.958429,3.436007e+04,9.259337e+06,10.405208,87.620173,16.051962,20.106720,56.343065,23.447108,35.886845,79.225614,3803.793389,1.658983,2.013422,0.532318,1.203378e+05,6.492729e+04,7695.264736,3.917936e+04,3.747583e+04,356.787528,8.721623,16.400178,2.010247e+05,26237.539756,29595.333399,768.171682,1531.045879,95.131846,40.050789,6699.356833,0.652165,3.285587,11.912373,0.772165,1904.607851,1693.773605,0.116406,8.914711,1.647457,2.272580,1.672274,6669.945981,7.750060,56.820107,4.394102e+07,0.186253,6.174358,3.029563e+04,6.240798e+04,494.626076,3442.880767,736.442666,0.503745,2.456476e+05,2.626716,0.165865,0.182836,0.155957
min,2012.000000,-5.429690e+05,0.000000,-1490.240000,0.000000,0.000000,1.810000,-1243.210000,-1.741950e+05,-3.575100e+04,-204.740000,-5300.390000,-318.880000,-307.410000,-705.810000,-653.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-9.121840e+05,-3.421598e+06,0.000000,-7.659680e+05,-7.716170e+05,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-127.980000,-100.000000,-98.900616,-1079.677183,-95.273372,-532450.000000,0.210000,91.815000,1.000000,-38.243959,-5511.340000,-5511.340000,-3.752854,-583.029412,0.296041,0.003003,0.012305,-195.502000,-0.004381,-99.338687,0.000000e+00,-4.745856,-273.924173,-1.665870e+05,-6.259930e+05,-10065.643303,-252600.000000,-46389.573529,-10.044103,-1.405336e+07,0.000004,0.000000,0.000000,0.000000
25%,2015.000000,3.754600e+04,6.660000,325.470000,12.622500,33.145000,76.800000,2.262500,-1.221500e+03,1.841995e+05,-1.367500,-7.620000,-3.960000,10.142500,-0.180000,-4.710000,4.852500,1.602500,4.260000,0.750000,1.190000,0.480000,-2.258500e+03,-1.259000e+03,302.250000,-3.560750e+03,-3.683500e+03,86.000000,0.680000,24.000000,5.637500e+02,0.000000,714.045000,-5.312766,-9.300412,-1.888081,-3.151108,-58.367810,0.510000,94.861000,16.000000,-0.026359,0.520215,0.039050,-0.014899,-0.020641,0.920303,0.200560,0.463008,0.000000

In [199]:
df.to_csv("./datasets/전처리준비/코스닥결측치처리완료.csv", encoding="utf-8-sig")